## Example: Particle Mesh Interpolation with multiple levels

### What Features Are We Using

* Mesh data
* Particle data
* Static mesh refinement

The examples so far used domain decomposition but no mesh refinement. This time, we alter the previous example
to work on hierarchy of meshes. We now have a `Vector<BoxArray>`, `Vector<DistributionMapping>`, `Vector<Geometry>`,
and a `Vector<IntVect>` telling us the refinement ratios between levels. We set up the grids to refine the central region of the domain with each level:

```C++
    Box domain = base_domain;
    IntVect size = IntVect(AMREX_D_DECL(parms.nx, parms.ny, parms.nz));
    for (int lev = 0; lev < parms.nlevs; ++lev)
    {
        ba[lev].define(domain);
        ba[lev].maxSize(parms.max_grid_size);
        dm[lev].define(ba[lev]);
        domain.grow(-size/4);   // fine level cover the middle of the coarse domain
        domain.refine(2);
    }
```

We again use "Cloud in cell" interpolation with randomly positioned particles. To handle particles near coarse-fine interfaces, we use the following algorithm. Particles with positions on the coarse level are deposited with the coarse cell spacing, and any mass that lands under a fine level is interpolated using piece-wise constant interpolation. Particles on the fine level are deposited with the fine cell spacing, and mass that "hangs off" the fine level is added to the coarse cell below it. 

The inputs parameters are the same as before, except you can now specify the number of levels to use as `nlevs`.

In [ ]:
import yt

In [ ]:
ds = yt.load("plt00000")

In [ ]:
sl = yt.SlicePlot(ds, 2, 'density')

In [ ]:
sl.annotate_grids()

In [ ]:
ad = ds.all_data()
ad.quantities.weighted_average_quantity('density', 'cell_volume')